In [2]:
import numpy as np
from torchvision.datasets import MNIST
def download_mnist(is_train: bool):
    dataset = MNIST(root='./data1',
        transform=lambda x: np.array(x).flatten(),
        download=True,
        train=is_train)
    
    mnist_data = []
    mnist_labels = []
    
    for image, label in dataset:
        mnist_data.append(image)
        mnist_labels.append(label)
        
    return mnist_data, mnist_labels
    
i_train_X, i_train_Y = download_mnist(True)
i_test_X, i_test_Y = download_mnist(False)

In [3]:
def convert_to_one_hot_encoding(labels: np.ndarray) -> np.ndarray:
    return np.eye(10)[labels]

def normalized(input: np.ndarray) -> np.ndarray:
    return input / 255

In [4]:
def transform_initial_data(train_X, train_Y, test_X, test_Y):
    train_Y = convert_to_one_hot_encoding(train_Y)
    test_Y = convert_to_one_hot_encoding(test_Y)
    
    train_X = np.array(train_X)
    test_X = np.array(test_X)

    train_X = normalized(train_X)
    test_X = normalized(test_X)
    
    return train_X, train_Y, test_X, test_Y

In [5]:
import math
def xavier_uniform(fan_in, fan_out):
    return math.sqrt(6/(fan_in + fan_out))

def he_uniform(fan_in, fan_out):
    return math.sqrt(6 / fan_in)

In [21]:
def initialize_weights_and_biases(neurons_per_layer):
    weights = []
    biases = []
    for i in range(1, len(neurons_per_layer)):
        fan_in, fan_out = neurons_per_layer[i - 1], neurons_per_layer[i]
        limit = he_uniform(fan_in, fan_out) #if i < len(neurons_per_layer) - 1 else he_uniform(fan_in, fan_out)
        
        weights.append(np.random.uniform(low=-limit, high=limit, size=(fan_in, fan_out)))
        biases.append(np.zeros((1, fan_out)))
        
    return weights, biases
    

In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z)) #-np.clip(z, -100, 100)

def sigmoid_prime(y):
    return y * (1 - y)

def tanh(z):
    return 2*sigmoid(2*z) - 1

def tanh_prime(y):
    return 1 - y*y

RELU_LEAK = 0.01

def leaky_relu(z):
    return np.where(z>0, z, z*RELU_LEAK)

def leaky_relu_prime(y):
    return np.where(y>0, 1, RELU_LEAK)


In [8]:
def softmax(z):
        e_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return e_z / e_z.sum(axis=1, keepdims=True)

In [9]:
activation_functions = {
        'leaky_relu': (leaky_relu, leaky_relu_prime),
        'sigmoid': (sigmoid, sigmoid_prime),
        'tanh': (tanh, tanh_prime)
    }

activation_function, activation_function_prime = activation_functions['leaky_relu']

In [10]:
def forward_propagation_train(inputs, weights, biases):
    output_per_layer = [inputs]
    DROPOUT_RATE = 0.1
    
    for i in range(len(weights)):
        w = weights[i]
        b = biases[i]
        x = output_per_layer[-1]
        
        z = x @ w + b

        #DROPOUT
        if i < len(weights) - 1:
            #z *= np.random.binomial(1, 1 - DROPOUT_RATE, z.shape)
            z *= np.random.choice([0, 1/(1-DROPOUT_RATE)], z.shape, p=[DROPOUT_RATE, 1-DROPOUT_RATE]) #[0, 1/(1-DROPOUT_RATE)]
            
        y = activation_function(z) if i < len(weights) - 1 else softmax(z)
        output_per_layer.append(y)
        
    return output_per_layer

def forward_propagation_test(inputs, weights, biases):
    output_per_layer = [inputs]
    
    for i in range(len(weights)):
        w = weights[i]
        b = biases[i]
        x = output_per_layer[-1]
        
        z = x @ w + b
        
        y = activation_function(z) if i < len(weights) - 1 else softmax(z)
        output_per_layer.append(y)
        
    return output_per_layer

In [11]:
def cross_entropy(prediction_output, train_output):
        return - np.sum(train_output * np.log(prediction_output))

In [23]:
def back_propagation(weights, biases, outputs_per_layer, labels, learning_rate):
        error = outputs_per_layer[-1] - labels
    
        REGULARIZAION_RATE = 0.01
        for i in reversed(range(len(weights))):
            y = outputs_per_layer[i]
            y_1 = outputs_per_layer[i+1]

            delta = error * (activation_function_prime(y_1) if i < len(weights) - 1 else 1)
            back_error = delta @ weights[i].T

            weights[i] -= learning_rate * (y.T @ delta + REGULARIZAION_RATE * weights[i]) / y.shape[0]
            biases[i] -= learning_rate * np.sum(delta, axis=0, keepdims=True) / y.shape[0]
    
            error = back_error
            

In [13]:
def predict(y):
    value_count = y.shape[1]
    y = np.argmax(y, axis=1)
    y = np.eye(value_count)[y]
    return y

def test_neural_network(test_input, weights, biases, test_output):
    fp = forward_propagation_test(test_input, weights, biases)
    predictions = predict(fp[-1])

    accuracy = np.mean(np.sum(predictions * test_output, axis=1))

    return accuracy

In [14]:
def update_learning_rate(current_learning_rate, accuracies, epoch, last_update_epoch):
    min_learning_rate = 1e-6
    patience = 7
    factor = 0.5
    
    if epoch - last_update_epoch >= patience:
        if len(accuracies) > patience and all(accuracies[-patience - 1] >= acc for acc in accuracies[-patience:]):
            new_learning_rate = max(current_learning_rate * factor, min_learning_rate)
            print(f"Reducing learning rate from {current_learning_rate} to {new_learning_rate}")
            return new_learning_rate, epoch
        
    return current_learning_rate, last_update_epoch

In [27]:
def train_neural_network(train_input, train_output, test_input, test_output):
    
    EPOCH_COUNT = 500
    BATCH_SIZE = 100

    train_count = train_input.shape[0]
    neurons_per_layer = [train_input.shape[1], 100, train_output.shape[1]]
    weights, biases = initialize_weights_and_biases(neurons_per_layer)

    accuracies_train = []
    accuracies_test = []
    learning_rate = 0.02
    last_update_epoch = -1

    for i in range(EPOCH_COUNT):
        indices = np.arange(0, train_count)
        np.random.shuffle(indices)

        train_X = train_input[indices]
        train_Y = train_output[indices]

        for j in range(0, train_count, BATCH_SIZE):
            batch_X = train_X[j:j + BATCH_SIZE]
            batch_Y = train_Y[j:j + BATCH_SIZE]

            outputs_per_layer = forward_propagation_train(batch_X, weights, biases)

            back_propagation(weights, biases, outputs_per_layer, batch_Y, learning_rate)

        acc_train = test_neural_network(train_input, weights, biases, train_output)
        acc_test = test_neural_network(test_input, weights, biases, test_output)
        accuracies_train.append(acc_train)
        accuracies_test.append(acc_test)

        learning_rate, last_update_epoch = update_learning_rate(learning_rate, accuracies_train, i, last_update_epoch)

        print(f"epoch {i+1}: test:{acc_test * 100: .2f}% ; train:{acc_train * 100: .2f}% ; learning_rate: {learning_rate}")

In [28]:
train_X, train_Y, test_X, test_Y = transform_initial_data(i_train_X, i_train_Y, i_test_X, i_test_Y)

train_neural_network(train_X, train_Y, test_X, test_Y)

epoch 1: test: 89.06% ; train: 88.04% ; learning_rate: 0.02
epoch 2: test: 91.00% ; train: 90.17% ; learning_rate: 0.02
epoch 3: test: 91.83% ; train: 91.22% ; learning_rate: 0.02
epoch 4: test: 92.44% ; train: 91.93% ; learning_rate: 0.02
epoch 5: test: 92.93% ; train: 92.53% ; learning_rate: 0.02
epoch 6: test: 93.28% ; train: 93.09% ; learning_rate: 0.02
epoch 7: test: 93.56% ; train: 93.44% ; learning_rate: 0.02
epoch 8: test: 93.81% ; train: 93.69% ; learning_rate: 0.02
epoch 9: test: 94.20% ; train: 94.07% ; learning_rate: 0.02
epoch 10: test: 94.48% ; train: 94.36% ; learning_rate: 0.02
epoch 11: test: 94.52% ; train: 94.57% ; learning_rate: 0.02
epoch 12: test: 94.85% ; train: 94.80% ; learning_rate: 0.02
epoch 13: test: 94.91% ; train: 94.98% ; learning_rate: 0.02
epoch 14: test: 95.13% ; train: 95.18% ; learning_rate: 0.02
epoch 15: test: 95.24% ; train: 95.31% ; learning_rate: 0.02
epoch 16: test: 95.36% ; train: 95.46% ; learning_rate: 0.02
epoch 17: test: 95.43% ; train: 9

KeyboardInterrupt: 